# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [1]:
import os
import sys

sys.path.append('..')
from utils import *

# Set your preferred name
USER = "Haoyang"
# Set the data and output directories
DATA_ROOT = '/media/techt/DATA'
OUTPUT_ROOT = '../../output1'
TRAIT = 'Epilepsy'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [2]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [3]:
dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:10]

['TCGA_Lower_Grade_Glioma_(LGG)',
 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)',
 'TCGA_Lung_Adenocarcinoma_(LUAD)',
 'TCGA_Lung_Cancer_(LUNG)',
 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)',
 'TCGA_Melanoma_(SKCM)',
 'TCGA_Mesothelioma_(MESO)',
 'TCGA_Ocular_melanomas_(UVM)',
 'TCGA_Ovarian_Cancer_(OV)',
 'TCGA_Pancreatic_Cancer_(PAAD)']

If no match is found, jump directly to GEO in Part 2.2

In [ ]:
trait_subdir = None
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.BRCA.sampleMap_BRCA_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.BRCA.sampleMap_HiSeqV2_PANCAN.gz')

In [ ]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [ ]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)
clinical_data_cols[:10]

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [ ]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

In [ ]:
candidate_age_cols = ['Age_at_Initial_Pathologic_Diagnosis_nature2012', 'age_at_initial_pathologic_diagnosis',
                      'days_to_birth', 'year_of_initial_pathologic_diagnosis']
candidate_gender_cols = ['Gender_nature2012', 'gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [ ]:
preview_df(clinical_data[candidate_age_cols])

In [ ]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [ ]:
preview_df(clinical_data[candidate_gender_cols])

In [ ]:
gender_col = 'gender'

In [ ]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col=age_col, gender_col=gender_col)

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [ ]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

In [ ]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort + '.csv'), index=False)

In [ ]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [4]:
dataset = 'GEO'
trait_subdir = "Epilepsy"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE156374', 'GSE157989', 'GSE182081', 'GSE182101', 'GSE199759', 'GSE205661']

Repeat the below steps for all the accession numbers

In [19]:
cohort = accession_num = "GSE205661"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/media/techt/DATA/GEO/Epilepsy/GSE205661/GSE205661_family.soft.gz',
 '/media/techt/DATA/GEO/Epilepsy/GSE205661/GSE205661-GPL13534_series_matrix.txt.gz')

### Initial filtering and clinical data preprocessing

In [20]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Integrated analysis of expression profile and potential pathogenic mechanism of temporal lobe epilepsy with hippocampal sclerosis"
!Series_summary	"To investigate the potential pathogenic mechanism of temporal lobe epilepsy with hippocampal sclerosis (TLE+HS), we have employed analyzing of the expression profiles of microRNA/ mRNA/ lncRNA/ DNA methylation in brain tissues of hippocampal sclerosis (TLE+HS) patients. Brain tissues of six patients with TLE+HS and nine of normal temporal or parietal cortices (NTP) of patients undergoing internal decompression for traumatic brain injury (TBI) were collected. The total RNA was dephosphorylated, labeled, and hybridized to the Agilent Human miRNA Microarray, Release 19.0, 8x60K. The cDNA was labeled and hybridized to the Agilent LncRNA+mRNA Human Gene Expression Microarray V3.0，4x180K. For methylation detection, the DNA was labeled and hybridized to the Illumina 450K Infinium Methylation BeadChip. The raw data was extracted from

In [21]:
clinical_data.head()

,!Sample_geo_accession,GSM6216198,GSM6216199,GSM6216200,GSM6216201,GSM6216202,GSM6216203,GSM6216204,GSM6216205,GSM6216206,GSM6216207,GSM6216208,GSM6216209,GSM6216210,GSM6216211,GSM6216212
0,!Sample_characteristics_ch1,tissue: Hippocampus,tissue: Hippocampus,tissue: Hippocampus,tissue: Hippocampus,tissue: Hippocampus,tissue: Hippocampus,tissue: Temporal lobe,tissue: Temporal lobe,tissue: Temporal lobe,tissue: Temporal lobe,tissue: Temporal lobe,tissue: Temporal lobe,tissue: Parietal lobe,tissue: Temporal lobe,tissue: Parietal lobe
1,!Sample_characteristics_ch1,gender: Female,gender: Male,gender: Male,gender: Female,gender: Female,gender: Male,gender: Female,gender: Male,gender: Female,gender: Male,gender: Male,gender: Male,gender: Male,gender: Male,gender: Male
2,!Sample_characteristics_ch1,age: 23y,age: 29y,age: 37y,age: 26y,age: 16y,age: 13y,age: 62y,age: 58y,age: 63y,age: 68y,age: 77y,age: 59y,age: 50y,age: 39y,age: 23y


In [22]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['tissue: Hippocampus', 'tissue: Temporal lobe', 'tissue: Parietal lobe'],
 1: ['gender: Female', 'gender: Male'],
 2: ['age: 23y',
  'age: 29y',
  'age: 37y',
  'age: 26y',
  'age: 16y',
  'age: 13y',
  'age: 62y',
  'age: 58y',
  'age: 63y',
  'age: 68y',
  'age: 77y',
  'age: 59y',
  'age: 50y',
  'age: 39y']}

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [23]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Epilepsy\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Epilepsy\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer.

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [24]:
is_gene_availabe = True
trait_row = 0
age_row = 2
gender_row = 1

trait_type = 'binary'

In [25]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [26]:
def convert_trait(tissue_type):
    """
    Convert tissue type to epilepsy presence (binary).
    Assuming epilepsy presence for 'Hippocampus' tissue.
    """
    if tissue_type == 'tissue: Hippocampus':
        return 1  # Epilepsy present
    else:
        return 0  # Epilepsy not present


def convert_age(age_string):
    """
    Convert age string to a numerical value.
    """
    if age_string.startswith('age: '):
        try:
            return int(age_string.split('age: ')[1].rstrip('y'))  # Extracting numeric part
        except ValueError:
            return None  # In case of parsing error
    else:
        return None  # If age string does not follow expected format


def convert_gender(gender_string):
    """
    Convert gender string to a binary representation.
    Female -> 0, Male -> 1, Unknown -> None
    """
    if gender_string == 'gender: Female':
        return 0
    elif gender_string == 'gender: Male':
        return 1
    else:
        return None  # In case of unknown gender

In [27]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

,GSM6216198,GSM6216199,GSM6216200,GSM6216201,GSM6216202,GSM6216203,GSM6216204,GSM6216205,GSM6216206,GSM6216207,GSM6216208,GSM6216209,GSM6216210,GSM6216211,GSM6216212
Epilepsy,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0
Age,23,29,37,26,16,13,62,58,63,68,77,59,50,39,23
Gender,0,1,1,0,0,1,0,1,0,1,1,1,1,1,1


### Genetic data preprocessing and final filtering

In [28]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM6216198,GSM6216199,GSM6216200,GSM6216201,GSM6216202,GSM6216203,GSM6216204,GSM6216205,GSM6216206,GSM6216207,GSM6216208,GSM6216209,GSM6216210,GSM6216211,GSM6216212
ID,,,,,,,,,,,,,,,
cg00000029,0.717,0.643,0.645,0.699,0.587,0.676,0.566,0.608,0.592,0.739,0.550,0.469,0.674,0.605,0.576
cg00000108,0.974,0.978,0.969,0.974,0.969,0.964,0.968,0.973,0.955,0.973,0.953,0.953,0.950,0.971,0.977
cg00000109,0.710,0.825,0.802,0.864,0.791,0.770,0.870,0.876,0.817,0.880,0.843,0.829,0.845,0.856,0.854
cg00000165,0.294,0.210,0.246,0.214,0.193,0.186,0.240,0.212,0.337,0.228,0.309,0.214,0.270,0.203,0.273
cg00000236,0.923,0.903,0.945,0.873,0.957,0.936,0.877,0.906,0.893,0.910,0.868,0.901,0.924,0.911,0.872


In [29]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['cg00000029',
 'cg00000108',
 'cg00000109',
 'cg00000165',
 'cg00000236',
 'cg00000289',
 'cg00000292',
 'cg00000321',
 'cg00000363',
 'cg00000622',
 'cg00000658',
 'cg00000714',
 'cg00000721',
 'cg00000734',
 'cg00000769',
 'cg00000807',
 'cg00000884',
 'cg00000905',
 'cg00000924',
 'cg00000948']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [30]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['cg00000029', 'cg00000108', 'cg00000109', 'cg00000165', 'cg00000236', 'cg00000289', 'cg00000292', 'cg00000321', 'cg00000363', 'cg00000622', 'cg00000658', 'cg00000714', 'cg00000721', 'cg00000734', 'cg00000769', 'cg00000807', 'cg00000884', 'cg00000905', 'cg00000924', 'cg00000948']\n"

If not required, jump directly to the gene normalization step

In [31]:
requires_gene_mapping = True

In [32]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

{'ID': ['cg00035864', 'cg00050873', 'cg00061679', 'cg00063477', 'cg00121626'], 'Name': ['cg00035864', 'cg00050873', 'cg00061679', 'cg00063477', 'cg00121626'], 'AddressA_ID': ['31729416', '32735311', '28780415', '16712347', '19779393'], 'AlleleA_ProbeSeq': ['AAAACACTAACAATCTTATCCACATAAACCCTTAAATTTATCTCAAATTC', 'ACAAAAAAACAACACACAACTATAATAATTTTTAAAATAAATAAACCCCA', 'AAAACATTAAAAAACTAATTCACTACTATTTAATTACTTTATTTTCCATC', 'TATTCTTCCACACAAAATACTAAACRTATATTTACAAAAATACTTCCATC', 'AAAACTAATAAAAATAACTTACAAACCAAATACTATACCCTACAACTCTC'], 'AddressB_ID': [nan, '31717405', nan, nan, nan], 'AlleleB_ProbeSeq': [nan, 'ACGAAAAAACAACGCACAACTATAATAATTTTTAAAATAAATAAACCCCG', nan, nan, nan], 'Infinium_Design_Type': ['II', 'I', 'II', 'II', 'II'], 'Next_Base': [nan, 'A', nan, nan, nan], 'Color_Channel': [nan, 'Red', nan, nan, nan], 'Forward_Sequence': ['AATCCAAAGATGATGGAGGAGTGCCCGCTCATGATGTGAAGTACCTGCTCAGCTGGAAAC[CG]AATTTGAGATAAATTCAAGGGTCTATGTGGACAAGACTGCTAGTGTCTCTCTCTGGATTG', 'TATCTCTGTCTGGCGAGGAGGCAACGCACAACTGTG

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [33]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we are analyzing a gene expression dataset, and find that its row headers are some identifiers related to genes:
    {gene_row_ids}
    To get the mapping from those identifiers to actual gene symbols, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the identifiers, and which key stores the gene symbols. Please strictly follow this format in your answer:
    identifier_key = key_name1
    gene_symbol_key = key_name2

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')


    As a biomedical research team, we are analyzing a gene expression dataset, and find that its row headers are some identifiers related to genes:
    ['cg00000029', 'cg00000108', 'cg00000109', 'cg00000165', 'cg00000236', 'cg00000289', 'cg00000292', 'cg00000321', 'cg00000363', 'cg00000622', 'cg00000658', 'cg00000714', 'cg00000721', 'cg00000734', 'cg00000769', 'cg00000807', 'cg00000884', 'cg00000905', 'cg00000924', 'cg00000948']
    To get the mapping from those identifiers to actual gene symbols, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the identifiers, and which key stores the gene symbols. Please strictly follow this format in your answer:
    identifier_key = key_name1
    gene_symbol_key = key_name2

    Gene annotation dictionary:
    {'ID': ['cg00035864', 'cg00050873', 'cg00061679', 'cg00063477', 'cg00121626'], 'Name': ['cg00035864', 'cg00050873', 'cg0006

In [34]:
if requires_gene_mapping:
    identifier_key = 'ID'
    gene_symbol_key = 'UCSC_RefGene_Name'
    gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [35]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

15 input query terms found dup hits:	[('ABCC13', 2), ('ABCC6P1', 2), ('ABCC6P2', 3), ('ADAM6', 3), ('AGAP11', 2), ('ALOX12P2', 2), ('ANKR
71 input query terms found no hit:	['A2BP1', 'A2LD1', 'AACSL', 'AARS', 'ABP1', 'ACCN1', 'ACCN2', 'ACCN3', 'ACCN4', 'ACCN5', 'ACN9', 'AC
6 input query terms found dup hits:	[('ATP6AP1L', 2), ('ATXN8OS', 2), ('BAGE2', 2), ('BMS1P1', 2), ('BMS1P4', 2), ('BRD7P3', 2)]
365 input query terms found no hit:	['ARNTL2', 'ARPM1', 'ARPP-21', 'ARSE', 'ASAM', 'ASAP1IT1', 'ASFMR1', 'ASNA1', 'ATHL1', 'ATP5A1', 'AT
3 input query terms found dup hits:	[('C2orf83', 2), ('C3P1', 2), ('C5orf60', 2)]
642 input query terms found no hit:	['C17orf56', 'C17orf57', 'C17orf59', 'C17orf60', 'C17orf61', 'C17orf62', 'C17orf63', 'C17orf64', 'C1
9 input query terms found dup hits:	[('CATSPER2P1', 2), ('CCDC144NL', 2), ('CCT6P1', 2), ('CDR1', 2), ('CECR7', 2), ('CELP', 2), ('CHKB-
101 input query terms found no hit:	['CBARA1', 'CBWD1', 'CBWD2', 'CBWD3', 'CBWD5', 'CBWD6', 'CCBL1', 'CC

In [36]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [37]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 15 samples.


In [38]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Epilepsy', the least common label is '1.0' with 6 occurrences. This represents 40.00% of the dataset.
The distribution of the feature 'Epilepsy' in this dataset is fine.

Quartiles for 'Age':
  25%: 24.5
  50% (Median): 39.0
  75%: 60.5
Min: 13.0
Max: 77.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 5 occurrences. This represents 33.33% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [39]:
if is_available:
    save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')
else:
    save_cohort_info(cohort, JSON_PATH, is_available)

In [40]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort + '.csv'), index=False)

### 3. Do regression & Cross Validation

In [41]:
# Check the information of usable cohorts
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
ranked_df

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,GSE205661,True,True,False,True,True,15,


In [42]:
# If both age and gender have available cohorts, select 'age' as the condition.
condition = 'Age'
filter_column = 'has_' + condition.lower()

condition_best_cohort, condition_ranked_df = filter_and_rank_cohorts(JSON_PATH, filter_column)
condition_best_cohort

'GSE205661'

In [43]:
condition_ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,GSE205661,True,True,False,True,True,15,


In [44]:
merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, condition_best_cohort + '.csv'))
merged_data.head()

,Epilepsy,Age,Gender,A1BG,A1CF,A2M,A2ML1,A4GALT,A4GNT,AAA1,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,1.0,23.0,0.0,0.498235,0.909286,0.803357,0.881167,0.454591,0.887167,0.9117,...,0.201900,0.039643,0.460750,0.5251,0.332605,0.554000,0.519077,0.365947,0.522561,0.162056
1,1.0,29.0,1.0,0.479765,0.900571,0.831571,0.874278,0.437136,0.895333,0.9172,...,0.208267,0.041286,0.236667,0.2783,0.323442,0.559333,0.517308,0.375368,0.512585,0.139722
2,1.0,37.0,1.0,0.490882,0.900000,0.814643,0.844056,0.437409,0.872500,0.9123,...,0.202633,0.041214,0.229333,0.3173,0.331791,0.558600,0.508000,0.368316,0.517439,0.149056
3,1.0,26.0,0.0,0.527765,0.900857,0.771929,0.903000,0.440500,0.804333,0.8902,...,0.187367,0.041000,0.448167,0.6069,0.332116,0.560267,0.528846,0.317737,0.517293,0.157000
4,1.0,16.0,0.0,0.485706,0.895857,0.776429,0.838000,0.438864,0.842500,0.8885,...,0.197133,0.047000,0.398750,0.4872,0.325698,0.546733,0.519769,0.356000,0.514829,0.158444


In [45]:
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [46]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [47]:
# Select appropriate models based on whether the dataset has batch effect.
# We experiment on two models for each branch. We will decide which one to choose later.

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [48]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
The cross-validation accuracy is 80.00% ± 26.67%


In [49]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

The cross-validation accuracy is 93.33% ± 13.33%


In [50]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

alpha for Lasso: 0.0003


### 4. Discussion and report

In [51]:
feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.3762
Found 14 genes with non-zero coefficients associated with the trait 'Epilepsy' conditional on the factor 'Age'. These genes are identified as significant based on the regression model.
Variable  Coefficient  Absolute Coefficient
     CA3   -11.813568             11.813568
   FAM3D    -8.039661              8.039661
 SLC23A2    -5.733602              5.733602
 RSPH10B    -4.606504              4.606504
  UBAP2L    -3.587676              3.587676
    RND2    -3.151997              3.151997
  OR10A4    -3.098253              3.098253
    CCL2    -2.942284              2.942284
   MMP28    -2.004953              2.004953
   OR6M1     1.947946              1.947946
  LINGO4    -1.499068              1.499068
    MRC2    -0.942940              0.942940
   ABCA7    -0.325261              0.325261
    EMP3    -0.117223              0.117223


/home/techt/AI4Science_gold_standard/utils.py:469: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df['Absolute Coefficient'] = gene_regression_df['Coefficient'].abs()


In [52]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.3762
Found 0 significant genes associated with the trait 'Epilepsy' conditional on the factor 'Age', with corrected p-value < 0.05:
Empty DataFrame
Columns: [Variable, Coefficient, p_value, corrected_p_value]
Index: []


/home/techt/AI4Science_gold_standard/utils.py:478: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
